# oloeriu bogdan: Advanced Lane Finding

do the necessery imports for the project

In [10]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

#this is my Camera class. 
import src.camera2 as camera

create a calibrated camera object and undistort an immage

In [13]:
calibrationPholder = "./camera_cal"
my_camera = camera.Camera(calibrationPholder)

TypeError: __init__() takes 1 positional argument but 2 were given